In [16]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import GithubFileLoader
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
load_dotenv()

True

In [ ]:
github_token = os.getenv("GITHUB_TOKEN")

#load files
loader = GithubFileLoader(
    repo="patelvandan11/Ai-Trip-Planner",
    branch="main",
    access_token=github_token,
    github_api_url="https://api.github.com",
    file_filter=lambda file_path: file_path.endswith((".py", ".md", ".jsx"))
 # Load only Python and Markdown files
)

documents = loader.load()

# print document sources
for doc in documents:
    print(f"\n--- {doc.metadata['source']} ---")
    print(doc.page_content)  # Print preview

embedding_function = NVIDIAEmbeddings()

vector_store = Chroma(
    collection_name="ai_trip_planner_docs",
    embedding_function=embedding_function,
    persist_directory="./chroma_langchain_db"
)
from langchain.text_splitter import RecursiveCharacterTextSplitter


splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(documents)

vector_store.add_documents(split_docs)

# Query 
query = input("Enter your question about the GitHub project: ")

retrieved_docs = vector_store.similarity_search(query, k=3)
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

#  LLM + Prompt + Chain
llm = ChatNVIDIA()   

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
        You are an expert AI assistant helping users understand a GitHub project based on its documentation.
        Use the following context, which is extracted from the project's README files and other Markdown documents.
        Be concise, accurate, and helpful. If the context does not contain a direct answer, say so clearly. Generate at least 4 sentences in your response.

        ---------------------
        Context:
        {context}
        ---------------------

        Question: {question}

        Answer:
            """.strip())


chain = prompt | llm | StrOutputParser()

# Run it
response = chain.invoke({
    "context": context,
    "question": query
})

print("\nAnswer from LLM:\n", response)


--- https://api.github.com/patelvandan11/Ai-Trip-Planner/blob/main/Backend/1.py ---
 

from dotenv import load_dotenv
load_dotenv()
import requests
from bs4 import BeautifulSoup
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import pandas as pd
import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"  # Replace with your actual key

# URL to scrape
url = ""
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Extract raw text content
text_content = soup.get_text(separator="\n", strip=True)

# Define the system prompt
system_msg = SystemMessage(content="""
You are a travel assistant AI. Extract and summarize the following details from the text:
- name (title of the package)
- category (Adventure, Nature, Heritage, Spiritual, etc.)
- highlights (top 3 attractions or experiences)
- duration (approximate in days)
- price (approximate in INR)
Return it as CSV line: name,category